In [ ]:
from pyspark.sql import SparkSession
from pysparkling import *
import h2o

ss = SparkSession.builder.getOrCreate()
hc = H2OContext.getOrCreate()

## Create H2OFrames

In [ ]:
adult_train_df = ss.read.parquet("../Data/spark-warehouse/adulttrain")
adult_train_h2o = hc.asH2OFrame(adult_train_df, "adult_train")

adult_valid_df = ss.read.parquet("../Data/spark-warehouse/adultvalid")
adult_valid_h2o = hc.asH2OFrame(adult_valid_df, "adult_valid")

## Set variables

In [ ]:
predictors = adult_valid_h2o.names[:]
response = "label"
predictors.remove(response)

## ML Algorithms

Added nfolds, fold_assignmen, keep_cross_validation_predictions

Only used training_fame for three algorithms

In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.xgboost import H2OXGBoostEstimator


model_gbm = H2OGradientBoostingEstimator(ntrees=50,
                                         max_depth=6,
                                         learn_rate=0.1,
                                         fold_assignment=fold_assignment,
                                         nfolds=n_folds,
                                         keep_cross_validation_predictions=keep_cross_validation_predictions,
                                         distribution="bernoulli")

model_dl = H2ODeepLearningEstimator(variable_importances=True,
                                    fold_assignment=fold_assignment,
                                    nfolds=n_folds,
                                    keep_cross_validation_predictions=keep_cross_validation_predictions,
                                    loss="Automatic")

model_xg = H2OXGBoostEstimator(fold_assignment=fold_assignment,
                               nfolds=n_folds,
                               keep_cross_validation_predictions=keep_cross_validation_predictions,
                              )

## Develop a stacked ensemble
http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/_modules/h2o/estimators/stackedensemble.html

http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/modeling.html#h2o.estimators.stackedensemble.H2OStackedEnsembleEstimator

In [ ]:
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
